## 네이버 영화평 감성분석 - LSTM

In [1]:
!pip install Konlpy > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [5]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


#### 1. 데이터 전처리
- train dataset

In [6]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [8]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [9]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [10]:
# 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- test dataset

In [11]:
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [12]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

#### 2. 텍스트 전처리
- train dataset

In [13]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [14]:
# 한글이 없는 글 --> ''만 남게됨
# ''만 남은 데이터는 제거: np.nan으로 대체후 dropna 실행
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [15]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test dataset

In [16]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

#### 3. 한글 형태소 분석


In [17]:
from konlpy.tag import Okt
okt = Okt()

In [18]:
from google.colab import files
up = files.upload()

Saving 한글불용어100.txt to 한글불용어100.txt


In [19]:
with open('한글불용어100.txt') as st:
    lines = st.readlines()

stop_words = [line.split('\t')[0] for line in lines]
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [20]:
from tqdm import tqdm

X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_train.append(clean_morph_review)

100%|██████████| 145393/145393 [06:44<00:00, 359.81it/s]


In [21]:
%%time
X_test = []
for review in test_df.document:
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_test.append(clean_morph_review)

CPU times: user 2min 3s, sys: 230 ms, total: 2min 3s
Wall time: 2min 4s


#### 4. Keras Tokenizer

In [22]:
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
t = Tokenizer()
t.fit_on_texts(X_train)
len(t.word_index)

43068

In [25]:
# 빈도수 상위 10,000개 단어로 인코딩
num_words = 10000
t = Tokenizer(num_words=num_words)
t.fit_on_texts(X_train)

In [26]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

In [27]:
# 데이터의 최대/평균 길이
max(len(s) for s in X_train), sum(len(s) for s in X_train) / len(X_train)

(67, 10.90946606782995)

In [28]:
# 한 문장의 최대 길이
max_len = 20

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [29]:
y_train = train_df.label.values
y_test = test_df.label.values

#### 5. LSTM 모델 정의/설정/학습

In [30]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [31]:
model = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    LSTM(128),
    Dense(1, 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best_naver_movie_lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [33]:
hist = model.fit(
    X_train, y_train, validation_split=0.2, epochs=30, batch_size=128, callbacks=[mc,es]
)

Epoch 1/30
909/909 [==============================] - ETA: 0s - loss: 0.4004 - accuracy: 0.8158
Epoch 1: val_loss improved from inf to 0.36354, saving model to best_naver_movie_lstm.h5
909/909 [==============================] - 97s 103ms/step - loss: 0.4004 - accuracy: 0.8158 - val_loss: 0.3635 - val_accuracy: 0.8396
Epoch 2/30
  1/909 [..............................] - ETA: 1:18 - loss: 0.3028 - accuracy: 0.8359

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


909/909 [==============================] - ETA: 0s - loss: 0.3239 - accuracy: 0.8580
Epoch 2: val_loss improved from 0.36354 to 0.35061, saving model to best_naver_movie_lstm.h5
909/909 [==============================] - 90s 99ms/step - loss: 0.3239 - accuracy: 0.8580 - val_loss: 0.3506 - val_accuracy: 0.8468
Epoch 3/30
909/909 [==============================] - ETA: 0s - loss: 0.2852 - accuracy: 0.8765
Epoch 3: val_loss did not improve from 0.35061
909/909 [==============================] - 91s 100ms/step - loss: 0.2852 - accuracy: 0.8765 - val_loss: 0.3576 - val_accuracy: 0.8458
Epoch 4/30
909/909 [==============================] - ETA: 0s - loss: 0.2524 - accuracy: 0.8920
Epoch 4: val_loss did not improve from 0.35061
909/909 [==============================] - 91s 100ms/step - loss: 0.2524 - accuracy: 0.8920 - val_loss: 0.3757 - val_accuracy: 0.8431
Epoch 5/30
909/909 [==============================] - ETA: 0s - loss: 0.2213 - accuracy: 0.9068
Epoch 5: val_loss did not improve from 

In [34]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

1527/1527 [==============================] - 21s 14ms/step - loss: 0.3562 - accuracy: 0.8424


[0.3562382459640503, 0.8423810601234436]